In [1]:
import pandas as pd
import folium
from folium import Marker
import geopandas as gpd
import os
from folium.plugins import HeatMap, MeasureControl
import folium.plugins
from IPython.display import display, clear_output
from plotly import express as px
import plotly.graph_objects as go
from shapely.geometry import Point
import numpy as np
import re
from time import sleep
from shapely.geometry import Polygon, MultiPolygon

---

# Carga de datos

In [2]:
data_path = os.path.join(os.path.dirname(os.getcwd()),  'data', 'pancho_raw')
output_data_path = os.path.join(os.path.dirname(os.getcwd()),  'data', 'pancho_clean')

listings = pd.read_csv(os.path.join(data_path, 'listings.csv'))
details = pd.read_csv(os.path.join(data_path, 'listings_detailed.csv'))
calendar = pd.read_csv(os.path.join(data_path, 'calendar.csv'))
gdf = gpd.read_file(os.path.join(data_path, 'neighbourhoods.geojson'))
crimes = pd.read_excel(os.path.join(data_path, 'crimenes.xlsx'), sheet_name='SEGURIDAD')
metro = pd.read_csv(os.path.join(data_path, 'metro.csv'))

---

# Limpieza

Los campos nulos de del dataset `listings` no son relevantes en nuestro análisis, por lo que no los tendremos en cuenta

In [3]:
metro['Longitude'] = metro['Longitude'].str.replace(',', '.').astype(float)
metro['Latitude'] = metro['Latitude'].str.replace(',', '.').astype(float)
metro['Traffic'] = metro['Traffic'].str.replace('.', '').astype(int)

---

# Preprocesado

## 1. Búsqueda de superficie
Debido a que no tenemos dentro del dataset el tamaño del alojamiento, vamos a buscarlo dentro del texto. Dado que el dataset está tanto en inglés como español, deberemos buscar:
- X m2
- X m²
- X square meters

In [4]:
common_columns = set(listings.columns).intersection(details.columns)
unique_columns_details = [col for col in details.columns if col not in common_columns]

listings = pd.merge(
	listings,
	details[['id'] + unique_columns_details],
	on='id',
	how='inner'
)

print(listings.columns)

listings['price'] = listings['price'].replace('[\$,]', '', regex=True).astype(float)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'listing_url', 'scrape_id', 'last_scraped',
       'description', 'neighborhood_overview', 'picture_url', 'host_url',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'accommodates', 'bathrooms', 'bathrooms_text',
       'bedrooms', 'beds', 'amenities', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_

<>:13: SyntaxWarning: invalid escape sequence '\$'
<>:13: SyntaxWarning: invalid escape sequence '\$'
/var/folders/x_/ghw0qsvj3p120vrgdzcvqpcc0000gn/T/ipykernel_24380/522814582.py:13: SyntaxWarning: invalid escape sequence '\$'
  listings['price'] = listings['price'].replace('[\$,]', '', regex=True).astype(float)


In [5]:
m2 = r'\b(\d+(?:\.\d+)?)\s*(?:m2|m²|metros cuadrados?|mts2|metros2)\b'
sq = r'\b(\d+(?:\.\d+)?)\s*(?:sq\s*ft|square\s*feet|ft²|sqft|feet²|sqfeet|sqf|square meters|sqm)\b'

listings['m2'] = listings['description'].str.extract(m2, flags=re.IGNORECASE)[0]
listings['sqft'] = listings['description'].str.extract(sq, flags=re.IGNORECASE)[0]

listings['m2'] = pd.to_numeric(listings['m2'], errors='coerce')
listings['sqft'] = pd.to_numeric(listings['sqft'], errors='coerce')

listings['m2'] = listings['m2'].fillna(listings['sqft'] * 0.092903)

listings.drop(columns=['sqft'], inplace=True)

## 2. Normalizar campos

In [6]:
calendar['date'] = pd.to_datetime(calendar['date'], errors='coerce')

listings['neighbourhood'] = listings['neighbourhood'].str.upper()
listings['neighbourhood_group'] = listings['neighbourhood_group'].str.upper()
gdf['neighbourhood'] = gdf['neighbourhood'].str.upper()
gdf['neighbourhood_group'] = gdf['neighbourhood_group'].str.upper()

In [7]:
crimes.head() ## -> Mover los datos hacía arriba

,SEGURIDAD CIUDADANA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,DISTRITOS,RELACIONADAS CON LAS PERSONAS,RELACIONADAS CON EL PATRIMONIO,POR TENENCIA DE ARMAS,POR TENENCIA DE DROGAS,POR CONSUMO DE DROGAS
2,CENTRO,33,711,2,22,1
3,ARGANZUELA,10,8,0,9,0
4,RETIRO,3,1,0,1,2


In [8]:
crimes.columns

Index(['SEGURIDAD CIUDADANA', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5'],
      dtype='object')

In [9]:
crimes.columns = crimes.loc[1, :]
print(crimes.columns)

Index(['DISTRITOS', 'RELACIONADAS CON LAS PERSONAS',
       'RELACIONADAS CON EL PATRIMONIO', 'POR TENENCIA DE ARMAS',
       'POR TENENCIA DE DROGAS', 'POR CONSUMO DE DROGAS'],
      dtype='object', name=1)


In [10]:
crimes.drop([0, 1, len(crimes) - 1], inplace=True)
crimes.reset_index(drop=True, inplace=True)
crimes.head()

1,DISTRITOS,RELACIONADAS CON LAS PERSONAS,RELACIONADAS CON EL PATRIMONIO,POR TENENCIA DE ARMAS,POR TENENCIA DE DROGAS,POR CONSUMO DE DROGAS
0,CENTRO,33,711,2,22,1
1,ARGANZUELA,10,8,0,9,0
2,RETIRO,3,1,0,1,2
3,SALAMANCA,4,36,2,6,0
4,CHAMARTÍN,10,12,0,23,5


## 3. Guardar los datos procesados

In [11]:
listings.to_csv(os.path.join(output_data_path, 'listings_merged.csv'), index=False)
calendar.to_csv(os.path.join(output_data_path, 'calendar.csv'), index=False)
crimes.to_csv(os.path.join(output_data_path, 'crimes.csv'), index=False)
gdf = gdf.to_crs(epsg=4326)
gdf.to_file(os.path.join(output_data_path, 'neighbourhoods.geojson'), driver='GeoJSON')
metro.to_csv(os.path.join(output_data_path, 'metro.csv'), index=False)